In [1]:
import numpy as np

In [2]:
## Forward Activations
def sigmoid(Z):
    return 1/(1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0,Z)

## Backward Activations
def sigmoid_backward(dA, Z):
    sigmoid_Z = sigmoid(Z)
    return dA * sigmoid_Z * (1 - sigmoid_Z)

def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True)
    dZ[Z <=0] = 0
    return dZ
    
## Initialize Layers
def initialize_layers(network_architecture):
    number_of_layers = len(network_architecture)
    parameters = {}
    
    for index, layer in enumerate(network_architecture):
        layer_index = index + 1
        layer_input_size = layer["input_dimension"]
        layer_output_size = layer["output_dimension"]
        
        ##### remove *0.1  to parameter initialization #####
        parameters['W' + str(layer_index)] = np.random.randn(layer_output_size, layer_input_size)*0.01
        parameters['b' + str(layer_index)] = np.random.randn(layer_output_size, 1)*0.01
        
    return parameters

## Forward Propagation

def single_layer_forward_propagation(A_previous, W_current, b_current, activation="relu"):
    Z_current = np.dot(W_current, A_previous) + b_current
    
    if activation is "relu":
        activation_function = relu
    elif activation is "sigmoid":
        activation_function = sigmoid
    else:
        raiseException('Unsupported activation')

    return activation_function(Z_current), Z_current

def full_forward_propagation(X, parameters, network_architecture):
    memory = {}
    A_current = X
    
    for index, layer in enumerate(network_architecture):
        layer_index = index + 1
        A_previous = A_current
        
        activation_function_current = layer["activation"]
        W_current = parameters["W" + str(layer_index)]
        b_current = parameters["b" + str(layer_index)]
        A_current, Z_current = single_layer_forward_propagation(A_previous, W_current, b_current, activation_function_current)

        memory["A" + str(index)] = A_previous
        memory["Z" + str(layer_index)] = Z_current
        
    return A_current, memory

## Loss
##### Check for correct implementation #####
def MSE(Y_hat, Y):
    loss = np.power(Y_hat - Y,2).mean()
    return loss

## Performance Metric
def l2_distance(Y_hat, Y):
    distance = np.sqrt(np.power(Y_hat - Y,2))
    return distance.mean()

## Backward Propagation
##### MIGHT BE CHANGED TO FOLLOW DISCUSSION'S IMPLEMENTATION #####
def single_layer_backward_propagation(dA_current, W_current, b_current, Z_current, A_previous, activation="relu"):
    m = A_previous.shape[1]
    
    if activation is "relu":
        backward_activation_function = relu_backward
    elif activation is "sigmoid":
        backward_activation_function = sigmoid_backward
    else:
        raise Exception('Unsupported activation')
        
    dZ_current = backward_activation_function(dA_current, Z_current)
    
    dW_current = np.dot(dZ_current, np.transpose(A_previous)) / m
    db_current = np.sum(dZ_current, axis=1, keepdims=True) / m
    dA_previous = np.dot(np.transpose(W_current), dZ_current)
    
    return dA_previous, dW_current, db_current

def full_backward_propagation(Y_hat, Y, memory, parameters, network_architecture):
    gradients = {}
    
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    
    dA_previous = - (np.divide(Y, Y_hat) - np.divide(1 - Y, 1 - Y_hat))
    
    for layer_index_previous, layer in reversed(list(enumerate(network_architecture))):
        layer_index_current = layer_index_previous + 1
        activation_function_current = layer["activation"]
        
        dA_current = dA_previous
        
        A_previous = memory["A" + str(layer_index_previous)]
        Z_current = memory["Z" + str(layer_index_current)]
        
        W_current = parameters["W" + str(layer_index_current)]
        b_current = parameters["b" + str(layer_index_current)]
        
        dA_previous, dW_current, db_current = single_layer_backward_propagation(dA_current, W_current, b_current, Z_current, A_previous, activation_function_current)
        
        gradients["dW" + str(layer_index_current)] = dW_current
        gradients["db" + str(layer_index_current)] = db_current
        
    return gradients

def update(parameters, gradients, network_architecture, learning_rate):
    
    for layer_index, layer in enumerate(network_architecture, 1):
        parameters["W" + str(layer_index)] -= learning_rate * gradients["dW" + str(layer_index)]
        parameters["b" + str(layer_index)] -= learning_rate * gradients["db" + str(layer_index)]
    
    return parameters

def train(X, Y, network_architecture, epochs, learning_rate):
    parameters = initialize_layers(network_architecture)
    loss_history = []
    accuracy_history = []
    
    for i in range(epochs):
        Y_hat, cache = full_forward_propagation(X, parameters, network_architecture)
        
        ##### loss should be MSE ######
        loss = MSE(Y_hat, Y)
        loss_history.append(loss)
        
        accuracy = l2_distance(Y_hat, Y)
        accuracy_history.append(accuracy)
        
        gradients = full_backward_propagation(Y_hat, Y, cache, parameters, network_architecture)
        
        parameters = update(parameters, gradients, network_architecture, learning_rate)
        
        print("Iteration: {:02} - loss: {:.5f} - l2-distance: {:.5f}".format(i+1, loss, accuracy))
        
    return parameters

In [3]:
## Network Architecture
network_architecture = [
    {"input_dimension":1, "output_dimension":64, "activation":"relu"},
    {"input_dimension":64, "output_dimension":64, "activation":"relu"},
    {"input_dimension":64, "output_dimension":1, "activation":"sigmoid"}
]

In [4]:
mean_text = input("Enter Mean:")
mean = float(mean_text)

standard_deviation_text = input("Enter Standard Deviation:")
standard_deviation = float(standard_deviation_text)

number_of_samples = 1000
epochs = 20
learning_rate = 0.1

#### Fix Dataset Generation - should be between [-2*std,2*std] #####
#### Recode 

X = np.random.uniform(mean - 2*standard_deviation, mean + 2*standard_deviation,(1000,1))
Y = 1/(standard_deviation*np.sqrt(2*np.pi)) * np.exp(-(X - mean)/(-2*np.square(standard_deviation)))

X_train = []
Y_train = []
X_test = []
Y_test = []

for i in range(number_of_samples):
    if i % 10 == 0:
        X_test.append(X[i])
        Y_test.append(Y[i])
    else:
        X_train.append(X[i])
        Y_train.append(Y[i])

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)


Enter Mean:0
Enter Standard Deviation:1


In [5]:
# Training
parameters = train(np.transpose(X_train), np.transpose(Y_train), network_architecture, epochs, learning_rate)

Iteration: 01 - loss: 0.06360 - l2-distance: 0.21726
Iteration: 02 - loss: 0.06354 - l2-distance: 0.21707
Iteration: 03 - loss: 0.06348 - l2-distance: 0.21688
Iteration: 04 - loss: 0.06343 - l2-distance: 0.21670
Iteration: 05 - loss: 0.06338 - l2-distance: 0.21653
Iteration: 06 - loss: 0.06334 - l2-distance: 0.21636
Iteration: 07 - loss: 0.06329 - l2-distance: 0.21620
Iteration: 08 - loss: 0.06325 - l2-distance: 0.21604
Iteration: 09 - loss: 0.06321 - l2-distance: 0.21588
Iteration: 10 - loss: 0.06317 - l2-distance: 0.21573
Iteration: 11 - loss: 0.06314 - l2-distance: 0.21559
Iteration: 12 - loss: 0.06310 - l2-distance: 0.21545
Iteration: 13 - loss: 0.06307 - l2-distance: 0.21531
Iteration: 14 - loss: 0.06304 - l2-distance: 0.21518
Iteration: 15 - loss: 0.06301 - l2-distance: 0.21505
Iteration: 16 - loss: 0.06298 - l2-distance: 0.21493
Iteration: 17 - loss: 0.06295 - l2-distance: 0.21481
Iteration: 18 - loss: 0.06293 - l2-distance: 0.21470
Iteration: 19 - loss: 0.06291 - l2-distance: 0

In [6]:
# Prediction
Y_test_hat, _ = full_forward_propagation(np.transpose(X_test), parameters, network_architecture)

# Accuracy achieved on the test set
acc_test = l2_distance(Y_test_hat, np.transpose(Y_test))
print("Test Set Distance: {:.5f}".format(acc_test))

Test Set Distance: 0.20998
